In [25]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
import os

import shutil


#----------------SPECIFY FILES TO READ IN ---------------------
start_file=1 #represent 1_pics e.g. start_file=end_file=1 only evaluates first folder!!!
end_file=1
name_new_folder="edited"
general_path = '../../savior_bags/'
dir_path = general_path+str(start_file)+"_pics"+"/"+name_new_folder
im_path = general_path+str(start_file)+"_pics"
#print dir_path

#CREATE NEW DIRECTORY
if os.path.exists(dir_path):
    shutil.rmtree(dir_path)
os.makedirs(dir_path)
#cv2.imwrite(dir+ '/' + str(i) + '.jpg',im1)

nummer=1

while(True):
    filename = im_path+ '/' + str(nummer) + '.jpg'
    im1 = cv2.imread(filename) #reads BGR
    if (im1 is None):
        #zum nächsten Ordner gehen!!!
        if (start_file>end_file):
            break
        else:
            start_file+=1
            if (start_file>end_file):
                break
            nummer=1
            dir_path = general_path+str(start_file)+"_pics"+"/"+name_new_folder
            im_path = general_path+str(start_file)+"_pics"
            #CREATE NEW DIRECTORY
            if os.path.exists(dir_path):
                shutil.rmtree(dir_path)
            os.makedirs(dir_path)
    
    
    else: #START MODIFYING THE IMAGE!!!
        #-------------HERE GOES THE REAL CODE-----------------------------------------------------------
        #-----------------------------------------------------------------------------------------------
        start = time.time()



        im1_cropped = im1[130:,:,:]
        im1_orig=np.copy(im1_cropped)
        #plt.imshow(im1_cropped); plt.show()

        #BREITE DER ENTE EVT AUSNUTZEN!!! oder evt: breite der breitesten stelle vs breite der schmalsten oder so^^
        # Convert BGR to HSV
        hsv = cv2.cvtColor(im1_cropped, cv2.COLOR_BGR2HSV)
        #plt.imshow(cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)); plt.show()

        #FILTER IMAGE
        lower_yellow = np.array([20,75,100])
        upper_yellow = np.array([40,255,255])
        # Threshold the HSV image to get only blue colors
        mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
        #plt.imshow(mask, cmap='gray'); plt.show()

        #evt pre process image through blurring,...
        blur2 = cv2.blur(mask,(8,4),0)
        super_threshold_indices = blur2 < 1
        blur2[super_threshold_indices] = 0 #wie in matlab da wo alle=1 werden sie auf 0 gesetzt!!!
        super_threshold_indices = blur2 >= 1
        blur2[super_threshold_indices] = 255
        #plt.imshow(blur2, cmap='gray'); plt.show()

        #--------------------SEGMENT AND EVALUATE-----------------------------------------------------------------
        #DIESES MAL UNTEN RECHTS ANFANGEN!!!!

        #ÜBERPRÜFUNG OB NACH GELBFILTER WAS ÜBRIGGEBLIEBEN IST:
        if(np.sum(mask)!=0):
            start = time.time()
            #IMAGE SEGMENTATION
            #bei mask die Randpixel auf null setzen sonst unten probleme mit den boundaries
            mask[0,:]=0
            mask[np.size(mask,0)-1,:]=0
            mask[:,0]=0
            mask[:,np.size(mask,1)-1]=0

            blur2[0,:]=0
            blur2[np.size(blur2,0)-1,:]=0
            blur2[:,0]=0
            blur2[:,np.size(blur2,1)-1]=0

            A=np.nonzero(mask)
            #A=np.nonzero(blur2)
            nonzero_val=np.empty(np.shape(A),dtype=int) #read the nonzero values
            nonzero_val[0,:]=A[0] 
            nonzero_val[1,:]=A[1]

            end_arr=np.size(nonzero_val,1)-1

            final=np.zeros_like(mask)
            i=1
            final[nonzero_val[0,end_arr],nonzero_val[1,end_arr]]=i
            #merged=0
            check = False

            #SEGMENTIERUNG MUSS DEFINITIV INTELLIGENTER WERDEN!!!! -> BISHER NOCH VIEL ZU VIELE!!!! SHIIIIIT^^
            for l in range(1,np.size(A,1)):
                j=end_arr-l

                if (final[nonzero_val[0,j]+1,nonzero_val[1,j]]!=0): #eins nach unten
                    final[nonzero_val[0,j],nonzero_val[1,j]]=final[nonzero_val[0,j]+1,nonzero_val[1,j]]
                    if (check and (final[nonzero_val[0,j],nonzero_val[1,j]]!=final[nonzero_val[0,j],nonzero_val[1,j]+1] and final[nonzero_val[0,j],nonzero_val[1,j]+1]!=0)):
                        #akt WERT final[nonzero_val[0,j],nonzero_val[1,j]] anders als der 1ns rechts final[nonzero_val[0,j],nonzero_val[1,j]+1] und der rechte !=0!!!
                        # -> der rechte WERT IST FALSCH!!! WEISSE DEN AKTUELLEN ZU!!
                        final[final == final[nonzero_val[0,j],nonzero_val[1,j]+1]] = final[nonzero_val[0,j],nonzero_val[1,j]] #
                        check = False 
                        #merged+=1


                elif (final[nonzero_val[0,j]+1,nonzero_val[1,j]+1]!=0): #eins rechts und unten
                    final[nonzero_val[0,j],nonzero_val[1,j]]=final[nonzero_val[0,j]+1,nonzero_val[1,j]+1]

                    if (check and (final[nonzero_val[0,j],nonzero_val[1,j]]!=final[nonzero_val[0,j],nonzero_val[1,j]+1] and final[nonzero_val[0,j],nonzero_val[1,j]+1]!=0)):
                        #akt WERT final[nonzero_val[0,j],nonzero_val[1,j]] anders als der 1ns rechts final[nonzero_val[0,j],nonzero_val[1,j]+1] und der rechte !=0!!!
                        # -> der rechte WERT IST FALSCH!!! WEISSE DEN AKTUELLEN ZU!!
                        final[final == final[nonzero_val[0,j],nonzero_val[1,j]+1]] = final[nonzero_val[0,j],nonzero_val[1,j]] #
                        check = False
                        #merged+=1


                if (final[nonzero_val[0,j]+1,nonzero_val[1,j]-1]!=0): #eins nach unten & links
                    final[nonzero_val[0,j],nonzero_val[1,j]]=final[nonzero_val[0,j]+1,nonzero_val[1,j]-1]
                    if (check and (final[nonzero_val[0,j],nonzero_val[1,j]]!=final[nonzero_val[0,j],nonzero_val[1,j]+1] and final[nonzero_val[0,j],nonzero_val[1,j]+1]!=0)):
                        #akt WERT final[nonzero_val[0,j],nonzero_val[1,j]] anders als der 1ns rechts final[nonzero_val[0,j],nonzero_val[1,j]+1] und der rechte !=0!!!
                        # -> der rechte WERT IST FALSCH!!! WEISSE DEN AKTUELLEN ZU!!
                        final[final == final[nonzero_val[0,j],nonzero_val[1,j]+1]] = final[nonzero_val[0,j],nonzero_val[1,j]] #
                        check = False
                        #merged+=1

                elif(final[nonzero_val[0,j],nonzero_val[1,j]+1]!=0): #eins nach rechts!
                    final[nonzero_val[0,j],nonzero_val[1,j]]=final[nonzero_val[0,j],nonzero_val[1,j]+1]
                    check = True

                else:
                    i+=1
                    final[nonzero_val[0,j],nonzero_val[1,j]]=i


            #print i-merged
            obst_arr=np.empty((4,i),dtype=int) #obstacle array: first two entries=center x and y, second=width third=height
            entry = 0;
            #plt.imshow(final); plt.show()


            #-----------------------FILTER MUSS TIEFENABHÄNGIG WERDEN!!!-------------------------------------------
            for k in range(1,i+1): #durch alle zahlen durchiterieren
                #first only keep large elements then eval their shape
                if (np.sum((final == k))<100): #skip all those who were merged away or have not enough pixels tiefenabh???
                    final[(final == k)]=0
                else:
                    #DIE BREITE MUSS ICH AUF DER GLEICHEN HÖHE NEHMEN!!!!!!
                    B=np.copy(final)
                    B[(B != k)]=0
                    C=np.nonzero(B)
                    ITER=np.reshape(C, (2,-1))
                    #print C 
                    top=np.min(C[0])
                    bottom=np.max(C[0])
                    left=np.min(C[1])
                    right=np.max(C[1])
                    height=bottom-top #da bottom höhere pixzahl hat!!
                    total_width=right-left

                    height_left=np.max(ITER[0,(C[1]==left)]) #ACHTUNG:kann mehrere WERTE HABEN
                    width_height_left = np.max(ITER[1,(C[0]==height_left)])
                    #WIDTH AT HEIGHT OF LEFT POSITION
                    width_left= width_height_left-left

                    height_right=np.max(ITER[0,(C[1]==right)]) #ACHTUNG:kann mehrere WERTE HABEN
                    width_height_right = np.min(ITER[1,(C[0]==height_right)])
                    #WIDTH AT HEIGHT OF RIGHT POSITION
                    width_right= right-width_height_right

                    #print width_right
                    #print width_left
                    #print width_right
                    #print height_left
                    #print height_right

                    #print "NEW OBJECT:"
                    #print bottom
                    #print width_right
                    #print 1.0*width_right/bottom

                    #WIDTH AT TOP
                    #MUSS NOCH ABFRAGE HIN DAMIT MAN NICHT OUT OF BOUNDS LÄUFT
                    #width_top = np.max(ITER[1,(C[0]==top+int(0.5*height))])-np.min(ITER[1,(C[0]==top+int(0.5*height))])

                    #if (abs(height_left-height_right)>10): #FILTER 1
                    #    final[(final == k)]=0
                    #if (abs(width_top-width_right)<20): #FILTER 2
                    #    final[(final == k)]=0

                    if (1.0*width_right/bottom<0.25): #FILTER 3
                        final[(final == k)]=0

                    else:
                        #ÜBERGABE?
                        #obst_arr[0,entry]=int(np.max(C[0])-0.5*width)
                        #obst_arr[1,entry]=int(np.max(C[1])-0.5*height)
                        #obst_arr[2,entry]=int(0.5*width)
                        #obst_arr[3,entry]=int(0.5*height)
                        entry+=1
                        cv2.rectangle(im1_cropped,(np.min(C[1]),np.min(C[0])),(np.max(C[1]),np.max(C[0])),(0,255,0),3)

            #eig box np.min breite und höhe!! if they passed the test!!!!
            #print abc
            end = time.time()
            #print(end - start)
            #plt.imshow(final); plt.show()
            last_img=np.concatenate((im1_orig[:,:,::-1],im1_cropped[:,:,::-1]),axis=1)
        #-----------------------------------------------------------------------------------------------
        #-------------------END OF REAL CODE  ----------------------------------------------------------
        
        else:
            last_img=np.concatenate((im1_orig[:,:,::-1],im1_cropped[:,:,::-1]),axis=1)
        
        #SAVE THE IMAGE
        conv = cv2.cvtColor(last_img, cv2.COLOR_RGB2BGR)
        cv2.imwrite(dir_path+ '/' + str(nummer) + '.jpg', conv)
        nummer+=1

print "FERTIG"

../../savior_bags/1_pics/edited
FERTIG
